In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
np.random.seed(132)
from functools import lru_cache

import sys

CODE_PATH = '../code'

sys.path.append(CODE_PATH)
import functions

from sklearn.metrics import accuracy_score, f1_score
from sklearn.datasets import load_iris, load_boston, load_breast_cancer, load_wine, load_digits
from scipy.optimize import minimize
from tqdm import tqdm

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [13]:
# dataset = load_breast_cancer()
dataset = load_digits()
df = pd.DataFrame(dataset['data'])
target = dataset['target']
# df = (df - df.mean())/(df.max() - df.min())
# df0 = df.copy()
# print(df.shape)
# print(target)
df.head()

# train/test split
from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(df.values, target, test_size=0.33, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(df.values, target, test_size=0.2, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=42)

print(X_train.shape, X_test.shape, X_val.shape)

(1005, 64) (360, 64) (432, 64)


# Обучаем классификаторы и строим матрицу

In [22]:
from sklearn.linear_model import LogisticRegression

l = np.unique(target).size
N = 30 # кол-во дихотомий
code_matrix = functions.make_random_dichs(l, N)

dich_classifiers = functions.train_dichs(code_matrix, X_train, y_train, 
                                         X_test, y_test, LogisticRegression)

Training dich classifiers: 100%|██████████| 30/30 [00:00<00:00, 48.00it/s]


In [23]:
for score_type in [None, 'accuracy', 'f1', 'confusion_list']:
    for weight_type in [None, 'accuracy', 'f1', 'confusion_list']:
        preds = functions.predict_all(X_val, dich_classifiers, code_matrix, score_type, weight_type)
        print(score_type, weight_type, accuracy_score(preds, y_val))

None None 0.8726851851851852
None accuracy 0.8796296296296297
None f1 0.875
None confusion_list 0.8726851851851852
accuracy None 0.8796296296296297
accuracy accuracy 0.8796296296296297
accuracy f1 0.8796296296296297
accuracy confusion_list 0.8796296296296297
f1 None 0.875
f1 accuracy 0.875
f1 f1 0.875
f1 confusion_list 0.875
confusion_list None 0.8888888888888888
confusion_list accuracy 0.8240740740740741
confusion_list f1 0.8240740740740741
confusion_list confusion_list 0.8240740740740741



# Конструируем модельную задачу

In [ ]:
np.random.seed(100)
cluster_objects = 100
num_clusters = 5

mean = np.zeros(2)
cov = np.eye(2) * 0.07
X = []

y = np.zeros(num_clusters**2*cluster_objects)

for i in np.linspace(0, num_clusters-1, num_clusters):
    for j in np.linspace(0, num_clusters-1, num_clusters):
        mean = np.array([i, j])
        X_cluster = np.random.multivariate_normal(mean, cov, cluster_objects)
        X.append(X_cluster)
X = np.vstack(X)
scatter(X[:,0], X[:,1], )

In [ ]:
# объединяем классы
classes_in_cluster = 5

order = np.array(range(num_clusters**2))
np.random.shuffle(order)
splits = np.split(order, 5)
for i, split in enumerate(splits):
    for item in split:
        y[item*cluster_objects:(item+1)*cluster_objects] = i

In [ ]:
print(y)
plot(y)

In [ ]:
figsize(8, 8)
scatter(X[:,0], X[:,1], c=y, )

In [ ]:
# модельная задача на 12 классов
colors = []
cs = []

# np.random.seed(100)
cluster_objects = 100
num_clusters = 5

mean = np.zeros(2)
cov = np.eye(2) * 0.07
X = []

y = np.zeros(num_clusters**2*cluster_objects)

for i in np.linspace(0, 4-1, 4):
    for j in np.linspace(0, 5-1, 5):
        c = np.random.randint(0, 12)
        mean = np.array([i, j])
        X_cluster = np.random.multivariate_normal(mean, cov, cluster_objects)
        X.append(X_cluster)
        colors += [c]*cluster_objects
        cs.append(c)
X = np.vstack(X)
print(cs)
scatter(X[:,0], X[:,1], c=colors)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.multioutput import ClassifierChain
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import jaccard_similarity_score
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import fetch_mldata

# Load a multi-label dataset
yeast = fetch_mldata('yeast')
X = yeast['data']
Y = yeast['target'].transpose().toarray()
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2,
                                                    random_state=0)

In [ ]:
yeast['target']

In [ ]:
df = pd.read_csv('../yeast.data.txt', sep=';', header=None)
X = df.values[:,1:-1]
y = pd.factorize(df[9])[0]
y

In [ ]:
from sklearn.manifold import TSNE


tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(X)

In [ ]:
scatter(tsne_results[:,0], tsne_results[:,1], c=y)
